### 11 November 2019
# RefSeq - Ensembl transcript region validation
### Pavlos Bousounis
***Last updated 11/17/2019***
___

### Load required modules

In [4]:
from datetime import datetime
import numpy as np
import os
import pandas as pd
import re

### Display current working directory and today's date

In [5]:
basedir = '/Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-11-11_RefSeq-Ensembl_GRCh37_validation'
os.chdir(basedir)
today = datetime.today().strftime('%Y-%m-%d')

print('Currently in {}\n'.format(basedir))
print('Today is: {}'.format(today))

Currently in /Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-11-11_RefSeq-Ensembl_GRCh37_validation

Today is: 2019-11-17


## Import the RefSeq transcript bed file
___

In [6]:
refseq_file = 'data/2019-11-17_RefSeq-GRCh37_latest_genomicGFF3_transcripts.bed'
refseq_colnames = ['chr', 'start', 'end', 'name_refseq']
refseq = pd.read_csv(refseq_file, sep='\t', header=None, 
                     names=refseq_colnames, low_memory=False).sort_values(by=['chr', 'start'])


print('\n')
refseq.info()
print('\n')
refseq.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 6929 entries, 0 to 6928
Data columns (total 4 columns):
chr            6929 non-null object
start          6929 non-null int64
end            6929 non-null int64
name_refseq    6929 non-null object
dtypes: int64(2), object(2)
memory usage: 270.7+ KB




,chr,start,end,name_refseq
0,1,11874,14409,NR_046018
1,1,14362,29370,NR_024540
2,1,1215816,1227409,NR_037668
3,1,1243960,1247057,NR_144369
4,1,1321091,1334715,NR_146722


## Import the Ensembl exon bed file
___

In [9]:
ensembl_file = 'data/2019-11-08_Ensembl-GRCh37_GFF3.bed'
ensembl_colnames = ['chr', 'start', 'end', 'name_ensembl']
ensembl = pd.read_csv(ensembl_file, sep='\t', header=None, 
                      names=ensembl_colnames, low_memory=False).sort_values(by=['chr', 'start'])


print('\n')
ensembl.info()
print('\n')
ensembl.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195408 entries, 0 to 1195407
Data columns (total 4 columns):
chr             1195408 non-null object
start           1195408 non-null int64
end             1195408 non-null int64
name_ensembl    1195408 non-null object
dtypes: int64(2), object(2)
memory usage: 45.6+ MB




,chr,start,end,name_ensembl
0,1,11869,12227,ENSE00002234944_ENST00000456328
3,1,11872,12227,ENSE00002234632_ENST00000515242
6,1,11874,12227,ENSE00002269724_ENST00000518655
10,1,12010,12057,ENSE00001948541_ENST00000450305
11,1,12179,12227,ENSE00001671638_ENST00000450305


## Merge the tables on chr, start, and end columns
___

In [10]:
# only exact chr, start, end coordinates matched
result = pd.merge(refseq, ensembl, how='inner', on=['chr', 'start', 'end']).sort_values(by=['chr', 'start'])


print('\n')
result.info()
print('\n')
result.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3609192 entries, 0 to 3609191
Data columns (total 5 columns):
chr             object
start           int64
end             int64
name_refseq     object
name_ensembl    object
dtypes: int64(2), object(3)
memory usage: 165.2+ MB




,chr,start,end,name_refseq,name_ensembl
0,1,11874,12227,DDX11L1-NR_046018:exon1(+),ENSE00002269724_ENST00000518655
1,1,12613,12721,DDX11L1-NR_046018:exon2(+),ENSE00003582793_ENST00000456328
2,1,12613,12721,DDX11L1-NR_046018:exon2(+),ENSE00003608237_ENST00000515242
3,1,13221,14409,DDX11L1-NR_046018:exon3(+),ENSE00002312635_ENST00000456328
4,1,14970,15038,WASH7P-NR_024540:exon2(-),ENSE00003497546_ENST00000541675


## Save RefSeq transcript accessions
___

In [20]:
refseq_tx_ids = pd.Series((refseq['name_refseq'].str.extract(r'(\w+-)(N(M|R)_\d+)')[1]).unique())

In [86]:
refseq_tx_ids_fname = '2019-11-11_RefSeq_accessions.tsv'
refseq_tx_ids.to_csv(refseq_tx_ids_fname, sep='\t', header=False, index=False)

In [85]:
refseq_tx_ids

0        NR_046018
1        NR_024540
2        NR_106918
3        NR_036051
4        NR_026818
           ...    
70329    NR_002195
70330    NR_001554
70331    NM_004680
70332    NM_170723
70333    NR_001524
Length: 70334, dtype: object

## Import UCSC Table Browser accession conversions
___

In [4]:
os.chdir('../2019-11-11_RefSeq2Ensembl_validation/')

In [51]:
rs2ens_file = 'data/2019-11-11_RefSeq-GRCh37_accession_conversion.txt'
rs2ens_colnames = ['refseq_acc', 'ensembl_acc']
rs2ens = pd.read_csv(rs2ens_file, sep='\t', comment='#', names=rs2ens_colnames)

print('\n')
rs2ens.info()
print('\n')
rs2ens.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74975 entries, 0 to 74974
Data columns (total 2 columns):
refseq_acc     74975 non-null object
ensembl_acc    41189 non-null object
dtypes: object(2)
memory usage: 1.1+ MB




,refseq_acc,ensembl_acc
0,NM_001308203,NaN
1,NM_001350217,NaN
2,NM_001350218,NaN
3,NM_032291,"ENST00000371037,ENST00000371039,ENST0000037103..."
4,NM_001350400,NaN


In [52]:
# drop NAs
rs2ens_nona = rs2ens.dropna()

print('\n')
rs2ens_nona.info()
print('\n')
rs2ens_nona.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 41189 entries, 3 to 74974
Data columns (total 2 columns):
refseq_acc     41189 non-null object
ensembl_acc    41189 non-null object
dtypes: object(2)
memory usage: 965.4+ KB




,refseq_acc,ensembl_acc
3,NM_032291,"ENST00000371037,ENST00000371039,ENST0000037103..."
5,NM_001145278,ENST00000337132
8,NM_032785,"ENST00000416121,ENST00000371839,"
9,NM_001145277,ENST00000337132
18,NM_018090,ENST00000337132


In [59]:
# Explode/Split column into multiple rows
tmp = pd.DataFrame(rs2ens_nona.ensembl_acc.str.split(',').tolist(), index=rs2ens_nona.refseq_acc).stack()
tmp = tmp.reset_index([0, 'refseq_acc']).drop_duplicates()
tmp.columns = ['refseq_acc', 'ensembl_acc']

print('\n')
tmp.info()
print('\n')
tmp.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 58692 entries, 0 to 60469
Data columns (total 2 columns):
refseq_acc     58692 non-null object
ensembl_acc    58692 non-null object
dtypes: object(2)
memory usage: 1.3+ MB




,refseq_acc,ensembl_acc
0,NM_032291,ENST00000371037
1,NM_032291,ENST00000371039
3,NM_032291,ENST00000435165
4,NM_032291,
5,NM_001145278,ENST00000337132


In [62]:
tmp['ensembl_acc'].replace('', np.nan, inplace=True)
r2e = tmp.dropna()

print('\n')
r2e.info()
print('\n')
r2e.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 48821 entries, 0 to 60469
Data columns (total 2 columns):
refseq_acc     48821 non-null object
ensembl_acc    48821 non-null object
dtypes: object(2)
memory usage: 1.1+ MB




,refseq_acc,ensembl_acc
0,NM_032291,ENST00000371037
1,NM_032291,ENST00000371039
3,NM_032291,ENST00000435165
5,NM_001145278,ENST00000337132
6,NM_032785,ENST00000416121


In [63]:
result['refseq_acc'] = result['name_refseq'].str.extract(r'(\w+-)(N(M|R)_\d+)')[1]
result.head()

,chr,start,end,name_refseq,name_ensembl,refseq_acc
0,1,11874,12227,DDX11L1-NR_046018:exon1(+),ENSE00002269724_ENST00000518655,NR_046018
1,1,12613,12721,DDX11L1-NR_046018:exon2(+),ENSE00003582793_ENST00000456328,NR_046018
2,1,12613,12721,DDX11L1-NR_046018:exon2(+),ENSE00003608237_ENST00000515242,NR_046018
3,1,13221,14409,DDX11L1-NR_046018:exon3(+),ENSE00002312635_ENST00000456328,NR_046018
4,1,14970,15038,WASH7P-NR_024540:exon2(-),ENSE00003497546_ENST00000541675,NR_024540


In [96]:
rs2ens = pd.merge(result, r2e, how='left', on='refseq_acc').sort_values(by=['chr', 'start'])
rs2ens_nona = rs2ens.dropna()
rs2ens_nona.loc[:, 'ensembl_exon'] = rs2ens_nona.loc[:, 'name_ensembl'].str.split('_').str[0]
rs2ens_nona.loc[:, 'ensembl_acc_alt'] = rs2ens_nona.loc[:, 'name_ensembl'].str.split('_').str[1]
rs2ens_nona.loc[:, 'gene'] = rs2ens_nona.loc[:, 'name_refseq'].str.split('-', 1).str[0]

/opt/anaconda3/envs/biopython/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/envs/biopython/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,chr,start,end,gene,ensembl_exon,ensembl_acc,ensembl_acc_alt,refseq_acc,name_refseq
0,1,11874,12227,DDX11L1,ENSE00002269724,ENST00000456328,ENST00000518655,NR_046018,DDX11L1-NR_046018:exon1(+)
1,1,12613,12721,DDX11L1,ENSE00003582793,ENST00000456328,ENST00000456328,NR_046018,DDX11L1-NR_046018:exon2(+)
2,1,12613,12721,DDX11L1,ENSE00003608237,ENST00000456328,ENST00000515242,NR_046018,DDX11L1-NR_046018:exon2(+)
3,1,13221,14409,DDX11L1,ENSE00002312635,ENST00000456328,ENST00000456328,NR_046018,DDX11L1-NR_046018:exon3(+)
4,1,14970,15038,WASH7P,ENSE00003497546,ENST00000438504,ENST00000541675,NR_024540,WASH7P-NR_024540:exon2(-)


In [98]:
refseq_ensembl = rs2ens_nona[['chr', 'start', 'end', 'gene', 'ensembl_exon', 'ensembl_acc', 'ensembl_acc_alt', 'refseq_acc', 'name_refseq']] 
refseq_ensembl.drop_duplicates().dropna()

print('\n')
refseq_ensembl.info()
print('\n')
refseq_ensembl.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 2616886 entries, 0 to 4326158
Data columns (total 9 columns):
chr                object
start              int64
end                int64
gene               object
ensembl_exon       object
ensembl_acc        object
ensembl_acc_alt    object
refseq_acc         object
name_refseq        object
dtypes: int64(2), object(7)
memory usage: 199.7+ MB




,chr,start,end,gene,ensembl_exon,ensembl_acc,ensembl_acc_alt,refseq_acc,name_refseq
0,1,11874,12227,DDX11L1,ENSE00002269724,ENST00000456328,ENST00000518655,NR_046018,DDX11L1-NR_046018:exon1(+)
1,1,12613,12721,DDX11L1,ENSE00003582793,ENST00000456328,ENST00000456328,NR_046018,DDX11L1-NR_046018:exon2(+)
2,1,12613,12721,DDX11L1,ENSE00003608237,ENST00000456328,ENST00000515242,NR_046018,DDX11L1-NR_046018:exon2(+)
3,1,13221,14409,DDX11L1,ENSE00002312635,ENST00000456328,ENST00000456328,NR_046018,DDX11L1-NR_046018:exon3(+)
4,1,14970,15038,WASH7P,ENSE00003497546,ENST00000438504,ENST00000541675,NR_024540,WASH7P-NR_024540:exon2(-)


In [99]:
refseq_ensembl_fileout = 'output/' + today + '_RefSeqGRCh37-EnsemblGRCh37_validation_table.tsv'
refseq_ensembl.to_csv(refseq_ensembl_fileout, sep='\t', index=False)

In [66]:
# rsdf = rs2ens[['chr', 'start', 'end', 'refseq_acc', 'ensembl_acc']]

# rsdf.head()

,chr,start,end,refseq_acc,ensembl_acc
0,1,11874,12227,NR_046018,ENST00000456328
1,1,12613,12721,NR_046018,ENST00000456328
2,1,12613,12721,NR_046018,ENST00000456328
3,1,13221,14409,NR_046018,ENST00000456328
4,1,14970,15038,NR_024540,ENST00000438504
